In [1]:
import re

import numpy as np
import pandas as pd

non_decimal = re.compile(r'[^\d.]+')

data_dir = '../data/'

In [2]:
columns = ['District', 'Route', 'County', 'Postmile', 'Description',
          'Back_Peak_Hourly', 'Back_Peak_Monthly', 'Back_AADT',
          'Ahead_Peak_Hourly', 'Ahead_Peak_Monthly', 'Ahead_AADT',
          'Back_Latitude', 'Back_Longitude', 'Ahead_Latitude', 'Ahead_Longitude', 'Year']

df_aadt = pd.DataFrame(columns=columns)

for year in range(2010, 2017):
    df_temp = pd.DataFrame.from_csv(data_dir + 'aadt/AADT%d.csv' % year, 
                                    index_col=None, header=0)
    
    df_temp.drop_duplicates(inplace=True)
    
    df_temp['Year'] = year
    
    drop_cols = [u'AbsPM_S_W,N,13,11', u'AbsPM_N_E,N,13,11']
    for col in drop_cols:
        if col in df_temp.columns:
            df_temp.drop([col], axis=1, inplace=True)

    df_temp.columns = columns

    df_aadt = df_aadt.append(df_temp)

df_aadt.Description.drop('BREAK IN ROUTE', inplace=True)

df_aadt.fillna(0, inplace=True)

df_aadt.rename_axis({'Ahead_Latitude': 'Latitude', 'Ahead_Longitude': 'Longitude'}, axis=1, inplace=True)
df_aadt.drop(['Back_Latitude', 'Back_Longitude'], axis=1, inplace=True)

df_aadt['Segment_ID'] = range(len(df_aadt))

In [3]:
seg_def_year = df_aadt.Year.max()

c1 = np.zeros((len(df_aadt[df_aadt.Year < seg_def_year]), 1))
c2 = np.array(range(1, len(df_aadt[df_aadt.Year == seg_def_year]) + 1))[:, np.newaxis]

df_aadt['Segment_Num'] = np.concatenate((c1, c2)).astype(np.int32)

In [4]:
print len(df_aadt)

df_aadt.head()

44289


,District,Route,County,Postmile,Description,Back_Peak_Hourly,Back_Peak_Monthly,Back_AADT,Ahead_Peak_Hourly,Ahead_Peak_Monthly,Ahead_AADT,Latitude,Longitude,Year,Segment_ID,Segment_Num
0,12.0,1,ORA,R0.129,"DANA POINT, JCT. RTE. 5",0,0.0,0.0,3750.0,40000.0,37000.0,33.467051,-117.669809,2010.0,0,0
1,12.0,1,ORA,8.43,"LAGUNA BEACH, MOUNTAIN",2850,38500.0,36000.0,2850.0,38500.0,36000.0,33.531752,-117.774872,2010.0,1,0
2,12.0,1,ORA,9.418,"LAGUNA BEACH, RTE. 133 N.",2850,38500.0,36000.0,3600.0,43000.0,40000.0,33.542738,-117.785319,2010.0,2,0
3,12.0,1,ORA,19.797,"NEWPORT BEACH, RTE. 55",4100,50000.0,46000.0,4400.0,42500.0,40000.0,33.621271,-117.929359,2010.0,3,0
4,12.0,1,ORA,21.549,SANTA ANA RIVER BRIDGE,3550,34000.0,32000.0,4300.0,41500.0,39000.0,33.630667,-117.956623,2010.0,4,0


In [5]:
def convert_to_int(x):
    try:
        return int(x)
    except:
        s = non_decimal.sub('', str(x))
        return int(s) if len(s) > 0 else 0

In [6]:
def convert_to_float(x):
    try:
        return float(x)
    except:
        s = non_decimal.sub('', str(x))
        return float(s) if len(s) > 0 else 0.0

In [7]:
df_aadt.District = df_aadt.District.apply(convert_to_int)
df_aadt.Year = df_aadt.Year.apply(convert_to_int)
df_aadt.Route = df_aadt.Route.apply(convert_to_int)
df_aadt.Postmile = df_aadt.Postmile.apply(convert_to_float)

df_aadt.Back_Peak_Hourly = df_aadt.Back_Peak_Hourly.apply(convert_to_int)

In [8]:
cols = [
    'Back_Peak_Hourly', 'Back_Peak_Monthly', 'Back_AADT',
    'Ahead_Peak_Hourly', 'Ahead_Peak_Monthly', 'Ahead_AADT',
]

for col in cols:
    df_aadt[col] = df_aadt[col].astype(np.int64)

In [9]:
print len(df_aadt)

df_aadt.head()

44289


,District,Route,County,Postmile,Description,Back_Peak_Hourly,Back_Peak_Monthly,Back_AADT,Ahead_Peak_Hourly,Ahead_Peak_Monthly,Ahead_AADT,Latitude,Longitude,Year,Segment_ID,Segment_Num
0,12,1,ORA,0.129,"DANA POINT, JCT. RTE. 5",0,0,0,3750,40000,37000,33.467051,-117.669809,2010,0,0
1,12,1,ORA,8.430,"LAGUNA BEACH, MOUNTAIN",2850,38500,36000,2850,38500,36000,33.531752,-117.774872,2010,1,0
2,12,1,ORA,9.418,"LAGUNA BEACH, RTE. 133 N.",2850,38500,36000,3600,43000,40000,33.542738,-117.785319,2010,2,0
3,12,1,ORA,19.797,"NEWPORT BEACH, RTE. 55",4100,50000,46000,4400,42500,40000,33.621271,-117.929359,2010,3,0
4,12,1,ORA,21.549,SANTA ANA RIVER BRIDGE,3550,34000,32000,4300,41500,39000,33.630667,-117.956623,2010,4,0


In [10]:
def get_df_segments(df):
    df_segments = {}
    
    # Start by looking only at segments for a given year
    for year in df.Year.unique():
        df_segments[year] = {}
        
        df_year = df[df.Year == year]
        
        # Then filter the collisions to their appropriate routes
        for route in df_year.Route.unique():
            df_segments[year][route] = {}

            df_route = df_year[df_year.Route == route]

            # Then filter them to each county to match with the Postmile
            for county in df_route.County.unique():
                df_segments[year][route][county] = df_route[df_route.County == county]

    return df_segments

In [11]:
df_segments = get_df_segments(df_aadt)

In [12]:
def get_postmile_boundary(row):
    # Grab identifying information to locate corresponding segment
    year = row.Year
    route = row.Route
    county = row.County        
    
    df_segment = df_segments[year][route][county]
    
    df_border = df_segment[df_segment.Postmile > row.Postmile]    
    
    return df_border.Postmile.min() if len(df_border) > 0 else 1000

In [13]:
df_aadt['Postmile_Boundary'] = df_aadt.apply(get_postmile_boundary, axis=1)

In [14]:
def get_postmile_distance(row):
    p1 = row.Postmile
    p2 = row.Postmile_Boundary
    
    if p2 < 1000:
        return p2 - p1
    
    year = row.Year
    route = row.Route
    county = row.County
    
    df_seg = df_aadt[(df_aadt.Year == year) 
                   & (df_aadt.Route == route) 
                   & (df_aadt.County == county)
                   & (df_aadt.Postmile_Boundary < 1000)]
    
    return (df_seg.Postmile_Boundary - df_seg.Postmile).mean()

In [15]:
df_aadt['Postmile_Distance'] = df_aadt.apply(get_postmile_distance, axis=1)

In [16]:
def get_nearest_segment(row):
    year = row.Year
    route = row.Route
    county = row.County
    
    if year == seg_def_year:
        return row.Segment_Num
    
    if not route in df_segments[seg_def_year] or not county in df_segments[seg_def_year][route]:
        return 0
    
    df_gps = df_segments[seg_def_year][route][county]
    
    df_postmile = df_gps[df_gps.Postmile == row.Postmile]
    
    if len(df_postmile) == 0:
        return 0
    
    gps = zip(df_postmile.Latitude - row.Latitude, df_postmile.Longitude - row.Longitude)
        
    gps_norm = np.linalg.norm(gps, axis=1)
    
    if gps_norm.min() > 1e-3:
        return 0
    
    i_min = gps_norm.argmin()
    
    return df_postmile.iloc[i_min].Segment_Num

In [17]:
df_aadt['Segment_Num'] = df_aadt.apply(get_nearest_segment, axis=1)

In [18]:
print df_aadt.columns

Index([u'District', u'Route', u'County', u'Postmile', u'Description',
       u'Back_Peak_Hourly', u'Back_Peak_Monthly', u'Back_AADT',
       u'Ahead_Peak_Hourly', u'Ahead_Peak_Monthly', u'Ahead_AADT', u'Latitude',
       u'Longitude', u'Year', u'Segment_ID', u'Segment_Num',
       u'Postmile_Boundary', u'Postmile_Distance'],
      dtype='object')


In [19]:
cols = [
    u'Segment_ID', u'Segment_Num', 
    u'Year', u'Route', u'County', u'District', 
    u'Postmile', u'Postmile_Boundary', u'Postmile_Distance',
    u'Latitude', u'Longitude', 
    u'Back_Peak_Hourly', u'Back_Peak_Monthly', u'Back_AADT',
    u'Ahead_Peak_Hourly', u'Ahead_Peak_Monthly', u'Ahead_AADT',
]

df_aadt = df_aadt[cols]
df_aadt.head()

,Segment_ID,Segment_Num,Year,Route,County,District,Postmile,Postmile_Boundary,Postmile_Distance,Latitude,Longitude,Back_Peak_Hourly,Back_Peak_Monthly,Back_AADT,Ahead_Peak_Hourly,Ahead_Peak_Monthly,Ahead_AADT
0,0,1,2010,1,ORA,12,0.129,8.430,8.301,33.467051,-117.669809,0,0,0,3750,40000,37000
1,1,4,2010,1,ORA,12,8.430,9.418,0.988,33.531752,-117.774872,2850,38500,36000,2850,38500,36000
2,2,5,2010,1,ORA,12,9.418,19.797,10.379,33.542738,-117.785319,2850,38500,36000,3600,43000,40000
3,3,11,2010,1,ORA,12,19.797,21.549,1.752,33.621271,-117.929359,4100,50000,46000,4400,42500,40000
4,4,13,2010,1,ORA,12,21.549,22.090,0.541,33.630667,-117.956623,3550,34000,32000,4300,41500,39000


In [20]:
df_aadt.to_csv(data_dir + 'df_aadt.csv')